In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import os
#per modelli statistici
import statsmodels.formula.api as sm
# Let us do the visualization imports:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# carico il dataset parziale
df = pd.read_csv('bank.csv', sep=";")
# df.dtypes

In [2]:
pandas_profiling.ProfileReport(df)


Number of variables,17
Number of observations,4521
Total Missing (%),0.0%
Total size in memory,600.5 KiB
Average record size in memory,136.0 B
Numeric,7
Categorical,10
Boolean,0
Date,0
Text (Unique),0
Rejected,0


   Input variables:
   # bank client data:
   1 - age (numeric)
   2 - job : type of job (categorical: "admin.","unknown","unemployed","management","housemaid","entrepreneur","student",
                                       "blue-collar","self-employed","retired","technician","services") 
   3 - marital : marital status (categorical: "married","divorced","single"; note: "divorced" means divorced or widowed)
   4 - education (categorical: "unknown","secondary","primary","tertiary")
   5 - default: has credit in default? (binary: "yes","no")
   6 - balance: average yearly balance, in euros (numeric) 
   7 - housing: has housing loan? (binary: "yes","no")
   8 - loan: has personal loan? (binary: "yes","no")
   # related with the last contact of the current campaign:
   9 - contact: contact communication type (categorical: "unknown","telephone","cellular") 
  10 - day: last contact day of the month (numeric)
  11 - month: last contact month of year (categorical: "jan", "feb", "mar", ..., "nov", "dec")
  12 - duration: last contact duration, in seconds (numeric)
   # other attributes:
  13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
  14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric, -1 means client was not previously contacted)
  15 - previous: number of contacts performed before this campaign and for this client (numeric)
  16 - poutcome: outcome of the previous marketing campaign (categorical: "unknown","other","failure","success")

  Output variable (desired target):
  17 - y - has the client subscribed a term deposit? (binary: "yes","no")

8. Missing Attribute Values: None

 # 0) PREPROCESSING:

In [192]:
# one_hot_encoding = pd.get_dummies(df)
df.head()
df.dtypes

# df.poutcome.value_counts()

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [202]:
#df.job.value_counts()
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [68]:
var_continue = ['age', 'balance', 'day', 'duration', 'campaign']
var_temporali = ['day', 'month']
var_dubbie = ['pdays']
var_dicotomiche = ['defaut', 'housing', 'loan']
var_categoriche = ['job','marital']

In [79]:
one_hot_encoding = pd.get_dummies(df)
grandezze = (one_hot_encoding.shape, df.shape)
grandezze

((4521, 53), (4521, 17))

In [77]:
# dfcovariates = (['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
#        'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
#        'previous', 'poutcome'])
# X = one_hot_encoding[dfcovariates]
# y = one_hot_encoding.y

# model = sm.Logit(y, X)
# result = model.fit()

# VOGLIO VEDERE L'EFFETTO DEL TIPO DI LAVORO SULL'OUTCOME:

In [200]:
import statsmodels.api as sm

onehotencoding_y = pd.get_dummies(df.y)
onehotencoding_job = pd.get_dummies(df.job)

y = pd.Series.to_frame(onehotencoding_y.yes) #hanno stipulato il contratto

X = onehotencoding_job
type(y)

#LINEAR MODEL
print("LINEAR MODEL")
#X = sm.add_constant(X)
est = sm.OLS(y, X)
est_linear = est.fit()
print(est_linear.summary())

#LOGIT MODEL
print("LOGIT MODEL")
est2 = sm.Logit(y,X)
est_logit = est2.fit()
print(est_logit.summary())
print("OR")
OR = np.exp(est_logit.params)
print(OR)
#EXSTRACT PROBABILITY
print("PROBABILITIES")
probabilities = OR/(1+OR)
print(probabilities)

LINEAR MODEL
                            OLS Regression Results                            
Dep. Variable:                    yes   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     6.352
Date:                Mon, 13 May 2019   Prob (F-statistic):           1.59e-10
Time:                        19:27:33   Log-Likelihood:                -1219.1
No. Observations:                4521   AIC:                             2462.
Df Residuals:                    4509   BIC:                             2539.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
admin.            0.1213      0.0

/home/dami/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/dami/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/dami/anaconda/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:3313: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull


In [185]:
df.job.value_counts()

df.groupby(['job', 'y']).size()



job            y  
admin.         no     420
               yes     58
blue-collar    no     877
               yes     69
entrepreneur   no     153
               yes     15
housemaid      no      98
               yes     14
management     no     838
               yes    131
retired        no     176
               yes     54
self-employed  no     163
               yes     20
services       no     379
               yes     38
student        no      65
               yes     19
technician     no     685
               yes     83
unemployed     no     115
               yes     13
unknown        no      31
               yes      7
dtype: int64

In [188]:
58/420
58/(420+58)

0.12133891213389121

# Scelgo le variabili per il modello logit

In [203]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

# Infilo tutte le variabili nella random forest